In [1]:
import pandas as pd
import numpy as np
import math
from derivatives import gradient, hessian
from descentMethods import max_descent, newton_descent_unrestricted
import time
import matplotlib.pyplot as plt
from scipy.optimize import minimize

## 1) Carga de datos

In [2]:
datos = pd.read_csv('/Users/pedrom2/Desktop/Personal/ITAM/Materias/Applied-Analysis/Proyecto 1/population-and-demography.csv')
datos

,Entity,Code,Year,Population - Sex: all - Age: all - Variant: estimates
0,Afghanistan,AFG,1950,7776182
1,Afghanistan,AFG,1951,7879343
2,Afghanistan,AFG,1952,7987783
3,Afghanistan,AFG,1953,8096703
4,Afghanistan,AFG,1954,8207953
...,...,...,...,...
18939,Zimbabwe,ZWE,2019,15271377
18940,Zimbabwe,ZWE,2020,15526888
18941,Zimbabwe,ZWE,2021,15797220
18942,Zimbabwe,ZWE,2022,16069061


In [3]:
datos_china = datos[datos['Entity'] == 'China']
datos_china = datos_china.drop(columns=['Entity','Code'])
datos_china = datos_china.rename(columns={'Population - Sex: all - Age: all - Variant: estimates':'Population'})
datos_china

,Year,Population
3330,1950,544044354
3331,1951,553758251
3332,1952,565131436
3333,1953,577557142
3334,1954,590113861
...,...,...
3399,2019,1423520354
3400,2020,1426106083
3401,2021,1426437276
3402,2022,1425179562


In [4]:
datos_mexico = datos[datos['Entity'] == 'Mexico']
datos_mexico = datos_mexico.drop(columns=['Entity','Code'])
datos_mexico = datos_mexico.rename(columns={'Population - Sex: all - Age: all - Variant: estimates':'Population'})
datos_mexico

,Year,Population
10952,1950,27587943
10953,1951,28320487
10954,1952,29090173
10955,1953,29899241
10956,1954,30753011
...,...,...
11021,2019,125762977
11022,2020,126799052
11023,2021,127648152
11024,2022,128613113


In [5]:
datos_china['Year'] = datos_china['Year'].apply(lambda x: (x-1949)/(10**6))
datos_china['Population'] = datos_china['Population'].apply(lambda x: x/(10**6))
datos_china

,Year,Population
3330,0.000001,544.044354
3331,0.000002,553.758251
3332,0.000003,565.131436
3333,0.000004,577.557142
3334,0.000005,590.113861
...,...,...
3399,0.000070,1423.520354
3400,0.000071,1426.106083
3401,0.000072,1426.437276
3402,0.000073,1425.179562


In [6]:
datos_mexico['Year'] = datos_mexico['Year'].apply(lambda x: (x-1949)/(10**6))
datos_mexico['Population'] = datos_mexico['Population'].apply(lambda x: x/(10**6))
datos_mexico

,Year,Population
10952,0.000001,27.587943
10953,0.000002,28.320487
10954,0.000003,29.090173
10955,0.000004,29.899241
10956,0.000005,30.753011
...,...,...
11021,0.000070,125.762977
11022,0.000071,126.799052
11023,0.000072,127.648152
11024,0.000073,128.613113


In [7]:
datos_china.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 3330 to 3403
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        74 non-null     float64
 1   Population  74 non-null     float64
dtypes: float64(2)
memory usage: 1.7 KB


In [8]:
datos_mexico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74 entries, 10952 to 11025
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Year        74 non-null     float64
 1   Population  74 non-null     float64
dtypes: float64(2)
memory usage: 1.7 KB


## 2) Función a minimizar

In [9]:
p0 = 27.587943 # México
#p0 = 544.044354 # China

def P(t, K, r):
    return K/(1+(K/p0-1)*np.exp(-1*r*t))

In [10]:
def f(x):
    suma = 0
    r = x[0]
    K = x[1]

    for index, row in datos_mexico.iterrows():
        y_k = row['Population']
        t_k = row['Year']
        suma += (P(t_k, K, r) - y_k) ** 2
    return suma/2

In [17]:
# Variables para medir tiempo
t_max = 0
t_newt = 0

# Selección del punto inicial xi (r, K)
xi = [.04, 210] # México
#xi = [.01, 1500] # China

start = time.time()
minimum_max, k_max = max_descent(f, xi)
end = time.time()
t_max = end-start

start = time.time()
minimum_newt, k_newt = newton_descent_unrestricted(f, xi, 13)
end = time.time()
t_newt = end-start

## 3) Resultados

In [18]:
g1 = np.linalg.norm(gradient(f, minimum_max))
g2 = np.linalg.norm(gradient(f, minimum_newt))

print(f'|===============|=======================|=========================|')
print(f'|===============|====Descenso Máximo====|=====Descenso Newton=====|')
print(f'|===============|=======================|=========================|')
print(f'   |∇f(x_min)|  |   {g1}  | {g2}  |')
print(f'   iteraciones  |          {k_max}          |            {k_newt}           |')
print(f'     cpu time   | {t_max}  |  {t_newt}  |')
print(f'{minimum_max}   |   {minimum_newt}')

|===============|=======================|=========================|
|===============|====Descenso Máximo====|=====Descenso Newton=====|
|===============|=======================|=========================|
   |∇f(x_min)|  |   5.067770905220642  | 4.517515199201777  |
   iteraciones  |          200          |            13           |
     cpu time   | 1.2697138786315918  |  0.30029988288879395  |
[946.46283293 370.32569911]   |   [ 68.02076914 202.65355261]
